# Active Learning Benchmarks and Classifier models

In [ ]:
from QueryStrategies.Random import uniform_random

In [ ]:
from QueryStrategies.Bayesian import max_entropy, bald

In [ ]:
from QueryStrategies.RL_based import RLAgent

In [ ]:
# from QueryStrategies.IRL_based import Q_Generator, Q_Sampler, 

In [ ]:
from Classifier.Model import vgg, CNN

# Packages

In [ ]:
import torch
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from torchvision.datasets import CIFAR10
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

from tqdm.autonotebook import tqdm
import os
from PIL import Image
import cv2
from subprocess import check_output
import time
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
import gym
from collections import deque
import seaborn as sns

from skorch import NeuralNetClassifier
from skorch import NeuralNet
from modAL.models import ActiveLearner

import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

Define some useful functions.

In [ ]:
def to_one_hot(y_tensor, ndims): 
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

def accuracy(out, yb):
    preds = torch.argmax(out, dim=-1)
    return (preds == yb).float().mean()

def show_image(X, y, index):
    candidate_image = X[index]  #X_train[1000,:]
    print("true label:" + str(y[index]))
    probs, _, _ = Classifier(X[index:index+1].float())
    print("predicted label:" + str(torch.argmax(probs, dim=-1)))
    candidate_image = candidate_image.reshape(28,28)
    plt.imshow(candidate_image, cmap='gray')
    plt.show()

# Data Preparation

Choices of data:
1- MNIST
2- CIFAR

## MNIST

In this section, we prepare the MNIST dataset. For CIFAR, run the codes in the following section.

In [ ]:
mnist_train = MNIST('.', train=True, download=True, transform=ToTensor())
mnist_test  = MNIST('.', train=False,download=True, transform=ToTensor())
traindataloader = DataLoader(mnist_train, shuffle=True, batch_size=60000)
testdataloader  = DataLoader(mnist_test , shuffle=True, batch_size=10000)
X, y = next(iter(traindataloader))
X_test , y_test  = next(iter(testdataloader))

print("Shape of features in training data: ", X.shape)
print("Shape of labels in training data: ", y.shape)
print("Shape of features in test data: ", X_test.shape)
print("Shape of labels in test data: ", y_test.shape)

In [ ]:
X_train, X_cross, y_train, y_cross = X[:50000], X[50000:], y[:50000], y[50000:]

X_train = X_train.reshape(50000, 1, 28, 28)
X_cross = X_cross.reshape(10000, 1, 28, 28)
X_test = X_test.reshape(10000, 1, 28, 28)

Let's save the data at the end so that we can use it later.

In [ ]:
# torch.save(X_train, 'Constructed_Data/X_train_MNIST.pt')
# torch.save(y_train, 'Constructed_Data/y_train_MNIST.pt')
# torch.save(X_cross, 'Constructed_Data/X_cross_MNIST.pt')
# torch.save(y_cross, 'Constructed_Data/y_cross_MNIST.pt')
# torch.save(X_test, 'Constructed_Data/X_test_MNIST.pt')
# torch.save(y_test, 'Constructed_Data/y_test_MNIST.pt')

## CIFAR

In this section, let's have alook at CIFAR dataset and prepare it for our experiments. If you use CIFAR, you have to adjust the shapes in the classifier accordingly.

In [ ]:
dataset = CIFAR10(root='data/', download=True, transform=ToTensor())
test_dataset = CIFAR10(root='data/', train=False, transform=ToTensor())
classes = dataset.classes
print("list of labels: ", classes)

In [ ]:
class_count = {}
for _, index in dataset:
    label = classes[index]
    if label not in class_count:
        class_count[label] = 0
    class_count[label] += 1
print("list of each class and its corresponding cardinality: \n", classes)

In [ ]:
torch.manual_seed(43)
val_size = 5000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds), len(test_dataset)

batch_size=128

traindataloader = DataLoader(train_ds, shuffle=True, batch_size=45000)
valdataloader = DataLoader(val_ds, shuffle=True, batch_size=5000)
testdataloader  = DataLoader(test_dataset , shuffle=True, batch_size=10000)
X_train, y_train = next(iter(traindataloader))
X_test , y_test  = next(iter(testdataloader))
X_cross , y_cross  = next(iter(valdataloader))

print("Shape of features in training data: ", X_train.shape)
print("Shape of labels in training data: ", y_train.shape)
print("Shape of features in test data: ", X_test.shape)
print("Shape of labels in test data: ", y_test.shape)

In [ ]:
for images, _ in traindataloader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

Let's save the data at the end so that we can use it later.

In [ ]:
# torch.save(X_train, 'Constructed_Data/X_train_CIFAR.pt')
# torch.save(y_train, 'Constructed_Data/y_train_CIFAR.pt')
# torch.save(X_cross, 'Constructed_Data/X_cross_CIFAR.pt')
# torch.save(y_cross, 'Constructed_Data/y_cross_CIFAR.pt')
# torch.save(X_test, 'Constructed_Data/X_test_CIFAR.pt')
# torch.save(y_test, 'Constructed_Data/y_test_CIFAR.pt')

## Training function for the classifier

In [ ]:
seed = 42 #For more repetitive results
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

#...... Set the budget in each episode and the number of episodes
budget = 500
num_episodes = 1

#...... Params for Classifier
output_shape_of_penultimate_layer = 128
num_classes = 10
lr_Classifier = 0.001

Classifier = CNN(output_shape_of_penultimate_layer, num_classes)
Classifier = Classifier.float()
optimizer_Classifier = optim.Adam(Classifier.parameters(), lr=lr_Classifier)
criterion = F.cross_entropy

In [ ]:
def train_Classifier(model, features, labels, optimizer, criterion=F.cross_entropy):
    
    optimizer.zero_grad()
    probs, logits, penult_out = model(features.float())
        
    loss_func = criterion    
    loss = loss_func(logits, labels)
    loss.backward()
    optimizer.step()
    
    return loss.detach().numpy()

## Setting

In [ ]:
#Creating a list of indices for initial rounds of play
initial_idx = np.array([],dtype=np.int)
for i in range(10):
    idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
    initial_idx = np.concatenate((initial_idx, idx))

X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

X_pool = np.delete(X_train, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)

print(initial_idx)
print(y_train[initial_idx])

In [ ]:
#Save the data
# torch.save(X_initial, 'Constructed_Data/X_initial.pt')
# torch.save(y_initial, 'Constructed_Data/y_initial.pt')
# torch.save(X_pool, 'Constructed_Data/X_pool.pt')
# torch.save(y_pool, 'Constructed_Data/y_pool.pt')
# np.save('Constructed_Data/initial_idx', np.array(initial_idx))

## Active Learning Procedure

It receives the query strategy, e.g., RL-based strategy, as the input and performs the active learning task using the given strategy.

In [ ]:
def active_learning_procedure(query_strategy,
                              X_cross,
                              y_cross,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              Classifier,
                              optimizer_Classifier,
                              criterion=F.cross_entropy,
                              n_queries=num_episodes*budget, #100
                              n_instances=1): #10
    
    loss = train_Classifier(Classifier, X_initial, y_initial, optimizer_Classifier, criterion)
    
    probs, _, _ = Classifier(X_cross.float())
    model_accuracy = accuracy(probs, y_cross)
    perf_hist = [model_accuracy]
    
    X_lab = X_initial
    y_lab = y_initial
    for index in range(n_queries):
        query_idx, query_instance = query_strategy(Classifier, X_pool, n_instances)

        X_lab = np.append(X_lab, X_pool[query_idx], axis=0)
        y_lab = np.append(y_lab, y_pool[query_idx], axis=0)
#         print(X_lab.shape)
#         print(y_lab.shape)
#         print(type(torch.from_numpy(X_lab)))
#         print(torch.from_numpy(X_lab).shape)
        
        loss = train_Classifier(Classifier, torch.from_numpy(X_lab), torch.from_numpy(y_lab), optimizer_Classifier, criterion) 
#         loss = train_Classifier(Classifier, X_pool[query_idx], y_pool[query_idx], optimizer_Classifier, criterion) #learner.teach(X_pool[query_idx], y_pool[query_idx])

        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        
        probs, _, _ = Classifier(X_cross.float())
        model_accuracy = accuracy(probs, y_cross)
        
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
    return perf_hist

## Running the benchmarks

We use different benchmark algorithms to perform active learning. To this end, they actively investigate each given unlablled data point and decide to ask for the corresponding label or not. The labelling budget is limited, which means that algorithms can ask for ground truth labels only for a certain number of data points.

In each case, we record the accuracy of the classifier over time.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

entropy_perf_hist = active_learning_procedure(max_entropy,
                                              X_cross,
                                              y_cross,
                                              X_pool,
                                              y_pool,
                                              X_initial,
                                              y_initial,
                                              Classifier,
                                              optimizer_Classifier,
                                              criterion,)

In [ ]:
# np.save('Results_Data/entropy_perf_hist', np.array(entropy_perf_hist))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

bald_perf_hist = active_learning_procedure(bald,
                                           X_cross,
                                           y_cross,
                                           X_pool,
                                           y_pool,
                                           X_initial,
                                           y_initial,
                                           Classifier,
                                           optimizer_Classifier,
                                           criterion,)

In [ ]:
# np.save('Results_Data/bald_perf_hist', np.array(bald_perf_hist))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

uniform_perf_hist = active_learning_procedure(uniform_random,
                                              X_cross,
                                              y_cross,
                                              X_pool,
                                              y_pool,
                                              X_initial,
                                              y_initial,
                                              Classifier,
                                              optimizer_Classifier,
                                              criterion,)


In [ ]:
# np.save('Results_Data/uniform_perf_hist', np.array(uniform_perf_hist))

In [ ]:
#...... Params for RL_based
epsilon = 0.1
EPSILON_DECAY = 0.1
MIN_EPSILON = 0.001
num_actions = 2
lr_RL = 1e-3
state_dim = output_shape_of_penultimate_layer + num_classes #state_shape[0], state_shape = (output_shape_of_penultimate_layer + num_classes, )
Q_hid_dim_1 = 256 #8
Q_hid_dim_2 = 128 #4
REPLAY_MEMORY_SIZE = 64 #To create an array with last "REPLAY_MEMORY_SIZE" steps for training
MIN_REPLAY_MEMORY_SIZE = 16 #Start training only if certain number of samples is already saved
MINIBATCH_SIZE = 64 #Not Used -- Batch from replay_memory
BATCH_SIZE = 32
UPDATE_TARGET_EVERY = 0 #To update target network. Target network is used to predict future_qs_list, model_RL is used to predict current_qs_list
DISCOUNT = 0.1

RL_Model = RLAgent(state_dim, num_actions, Q_hid_dim_1, Q_hid_dim_2, 
                   REPLAY_MEMORY_SIZE, MIN_REPLAY_MEMORY_SIZE, BATCH_SIZE, MINIBATCH_SIZE, DISCOUNT, UPDATE_TARGET_EVERY)


For RL-based approach, we need to re-define the active learning function as follows.

In [ ]:
#AL With Function
def active_learning_procedure_RL(RL_Model,
                              X_cross,
                              y_cross,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              Classifier,
                              optimizer_Classifier,
                              n_episodes=num_episodes, 
                              epsilon=epsilon,
                              MIN_EPSILON=MIN_EPSILON,
                              EPSILON_DECAY=EPSILON_DECAY,
                              penult_len=output_shape_of_penultimate_layer,
                              num_classes=10,
                              num_actions=2,
                              criterion=F.cross_entropy,
                              n_queries=budget, #100
                              n_instances=1, #10
                              random_selection=False): 

    loss = train_Classifier(Classifier, X_initial, y_initial, optimizer_Classifier, criterion)
    
    probs, _, _ = Classifier(X_cross.float())
    model_accuracy = accuracy(probs, y_cross)
    perf_hist = [model_accuracy]
    
    rew = []
    saver_count = False
    X_lab = X_initial
    y_lab = y_initial
    t = -1
    for episode in tqdm(range(0, int(n_episodes)), ascii=True, unit='episodes'):
        step = 0
        done = False
        while not done:
            
            t = t+1
            print(t)
            idx = np.random.choice(range(len(X_pool)), size=n_instances, replace=False)

            #...... Receive the Current state
            probs, _, penult_out = Classifier(X_pool[idx].float())
            penult_out = penult_out.view(penult_len)
            probs = probs.view(num_classes)
            state_at_t = torch.cat((penult_out, probs), -1)
            
            probs, _, _ = Classifier(X_cross.float())
            model_accuracy_before = accuracy(probs, y_cross)
    
            #RL picks an action to label/not label #query_idx, query_instance = query_strategy(Classifier, X_pool, n_instances)
            Q_values = RL_Model.get_qs(state_at_t.detach().numpy())
            action_at_t = np.argmax(Q_values) # Get action from Q table

            if random_selection:
                if np.random.random() > epsilon:
                    action_at_t = action_at_t
                else: # Get random action
                    action_at_t = np.random.randint(0, num_actions) # Get random action: 2 possible choices: i) 1 == label ii) 0 == do not label
        
            if action_at_t == 1:
                step = step + 1
                
                X_lab = np.append(X_lab, X_pool[idx], axis=0)
                y_lab = np.append(y_lab, y_pool[idx], axis=0)

                loss = train_Classifier(Classifier, torch.from_numpy(X_lab), torch.from_numpy(y_lab), optimizer_Classifier, criterion) 
                
#                 X_pool = np.delete(X_pool, idx, axis=0)
#                 y_pool = np.delete(y_pool, idx, axis=0)

                probs, _, _ = Classifier(X_cross.float())
                model_accuracy_after = accuracy(probs, y_cross)
                
                print('Accuracy after query {n}: {acc:0.4f}'.format(n=step, acc=model_accuracy_after))
                perf_hist.append(model_accuracy_after)
        
            else:
                model_accuracy_after = model_accuracy_before
                
            reward_at_t = model_accuracy_after - model_accuracy_before #Compute reward
            rew.append(reward_at_t)

            #...... Receive the New state
            probs, _, penult_out = Classifier(X_pool[idx].float())
            penult_out = penult_out.view(penult_len)
            probs = probs.view(num_classes)
            state_at_next_t = torch.cat((penult_out, probs), -1)

            if step >= budget: #Update the step and check episode using budget
                done = True

            replay_memory_DataColl = RL_Model.update_replay_memory_DataColl((state_at_t.detach().numpy(), action_at_t)) #Add Later: , log_q_tau_var_D, 1

            if episode == n_episodes-1 and step == budget:
                print("episode before saving:" + str(episode))
                print("step before saving:" + str(step))
                saver_count = True

            #...... Updating the RL model (training the weights of RL model),  Every step we update replay memory and train main network
            RL_Model.update_replay_memory((state_at_t.detach().numpy(), action_at_t, reward_at_t.detach().numpy(), state_at_next_t.detach().numpy(), done))
            RL_Model.train(done, step, saver_count) #train(self, terminal_state, step, saver_count)

            X_pool = np.delete(X_pool, idx, axis=0)
            y_pool = np.delete(y_pool, idx, axis=0)
            
        if random_selection: #Decay epsilon
            if epsilon > MIN_EPSILON:
                epsilon *= EPSILON_DECAY
                epsilon = max(MIN_EPSILON, epsilon)

    return perf_hist, replay_memory_DataColl, rew

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

RL_perf_hist, RL_replay_memory_DataColl, RL_rew = active_learning_procedure_RL(RL_Model,
                                              X_cross,
                                              y_cross,
                                              X_pool,
                                              y_pool,
                                              X_initial,
                                              y_initial,
                                              Classifier,
                                              optimizer_Classifier,
                                              n_episodes=num_episodes, 
                                              epsilon=epsilon,
                                              MIN_EPSILON=MIN_EPSILON,
                                              EPSILON_DECAY=EPSILON_DECAY,
                                              penult_len=output_shape_of_penultimate_layer,
                                              num_classes=10,
                                              num_actions=2,
                                              criterion=F.cross_entropy,
                                              n_queries=budget, #100
                                              n_instances=1, #10
                                              random_selection=False)


In [ ]:
# IRL_perf_hist = np.load('Results_Data/perf_hist_IRL_4.npy')

## Comparing the Results

We compare the accuracy trend of the classifier over time for different query strategies as new labelled data points are fed to update the model.

In [ ]:
# sns.set()
# sns.set_style("white")
fig = plt.figure(figsize=[8, 5])
plt.plot(np.arange(120)[::20], 100*entropy_perf_hist[::20], color = 'blue', marker = "d", markersize = 5, fillstyle='none', markeredgewidth=1.5, label="Max Entropy")
plt.plot(np.arange(120)[::20], 100*uniform_perf_hist[:120][::20], color = 'black', marker = "s", markersize = 5, fillstyle='none', markeredgewidth=1.5, label="Random")
# plt.plot(np.arange(120)[::20], 100*perf_hist_IRL[::20], color = 'orange', marker = "o", markersize = 5, fillstyle='none', markeredgewidth=1.5, label="Our Method")
plt.plot(np.arange(120)[::20], 100*RL_perf_hist[::20], color = 'green', marker = ">", markersize = 5, fillstyle='none', markeredgewidth=1.5, label="RL")

plt.legend(loc = 'lower right', fontsize = 20)
# plt.title(f"Accuracy vs. Num Labelled data")
plt.xlabel("#Labelled Data", fontsize = 22)
plt.ylabel("Accuracy", fontsize = 22)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
# plt.grid()
plt.show()



In [ ]:
# fig.savefig("Results_Before_Transfer_to_Test_Phase.png", format = 'png', dpi = 300)